# Data Preparation

## Objective
The objective of this notebook is to prepare the online retail dataset for clustering analysis. The steps include loading, exploring, cleaning, engineering features, transforming the data, and finally saving the cleaned dataset. This data preparation pipeline will help identify customer segments using unsupervised learning techniques like K-means clustering.


In [1]:
import numpy as np
import pandas as pd

## 1. Loading Data

In [2]:
df=pd.read_excel(r"C:\Users\HP\Downloads\Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## 2. Exploring Data

In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
None


In [5]:
# Statistical summary (for numerical columns)
print(df.describe())

            Quantity                    InvoiceDate      UnitPrice  \
count  541909.000000                         541909  541909.000000   
mean        9.552250  2011-07-04 13:34:57.156386048       4.611114   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       218.081158                            NaN      96.759853   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


In [6]:
# Check missing values per column
print(df.isnull().sum())

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


## 3. Cleaning Data

In [14]:
# Dropping rows with missing 'Description'
df_cleaned = df.dropna(subset=['Description']).copy()

# Check missing values again
print(df_cleaned.isnull().sum())


InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64


In [15]:
# Fill missing CustomerID with 'Unknown'
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].fillna('Unknown')


In [16]:
# Remove duplicate entries
df_cleaned = df_cleaned.drop_duplicates()

# Confirm number of rows after removing duplicates
print(f"Remaining rows after duplicate removal: {df_cleaned.shape[0]}")


Remaining rows after duplicate removal: 535187


In [17]:
# Convert InvoiceDate to datetime
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

# Ensure CustomerID is string
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].astype(str)

# Check updated data types
print(df_cleaned.dtypes)


InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object


## 4. Feature Engineering

In [18]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Scale 'Quantity' and 'UnitPrice'
df_cleaned[['Quantity_scaled', 'UnitPrice_scaled']] = scaler.fit_transform(df_cleaned[['Quantity', 'UnitPrice']])

# View the scaled columns
print(df_cleaned[['Quantity', 'Quantity_scaled', 'UnitPrice', 'UnitPrice_scaled']].head())


   Quantity  Quantity_scaled  UnitPrice  UnitPrice_scaled
0         6        -0.016761       2.55         -0.021520
1         6        -0.016761       3.39         -0.012892
2         8        -0.007631       2.75         -0.019465
3         6        -0.016761       3.39         -0.012892
4         6        -0.016761       3.39         -0.012892


In [19]:
# One-Hot Encoding for 'Country' (drop_first=True to avoid dummy trap)
df_encoded = pd.get_dummies(df_cleaned, columns=['Country'], drop_first=True)

# View encoded dataframe shape
print(f"Shape after encoding: {df_encoded.shape}")


Shape after encoding: (535187, 46)


In [20]:
# Creating TotalAmount feature
df_encoded['TotalAmount'] = df_encoded['Quantity'] * df_encoded['UnitPrice']

# View sample
print(df_encoded[['Quantity', 'UnitPrice', 'TotalAmount']].head())


   Quantity  UnitPrice  TotalAmount
0         6       2.55        15.30
1         6       3.39        20.34
2         8       2.75        22.00
3         6       3.39        20.34
4         6       3.39        20.34


In [21]:
# Extract YearMonth for time-based analysis
df_encoded['YearMonth'] = df_encoded['InvoiceDate'].dt.to_period('M')

# View sample
print(df_encoded[['InvoiceDate', 'YearMonth']].head())


          InvoiceDate YearMonth
0 2010-12-01 08:26:00   2010-12
1 2010-12-01 08:26:00   2010-12
2 2010-12-01 08:26:00   2010-12
3 2010-12-01 08:26:00   2010-12
4 2010-12-01 08:26:00   2010-12


In [22]:
# Extract Hour from InvoiceDate
df_encoded['InvoiceHour'] = df_encoded['InvoiceDate'].dt.hour

# View sample
print(df_encoded[['InvoiceDate', 'InvoiceHour']].head())


          InvoiceDate  InvoiceHour
0 2010-12-01 08:26:00            8
1 2010-12-01 08:26:00            8
2 2010-12-01 08:26:00            8
3 2010-12-01 08:26:00            8
4 2010-12-01 08:26:00            8


In [23]:
# Compute correlation matrix
corr_matrix = df_encoded.corr(numeric_only=True)

# Correlation of all features with TotalAmount
print(corr_matrix['TotalAmount'].sort_values(ascending=False))


TotalAmount                     1.000000
Quantity                        0.888196
Quantity_scaled                 0.888196
Country_Netherlands             0.017831
Country_Japan                   0.005467
Country_Sweden                  0.004713
Country_EIRE                    0.004565
Country_Denmark                 0.002128
Country_Germany                 0.001829
Country_Norway                  0.001681
Country_France                  0.001647
Country_Switzerland             0.001619
Country_Finland                 0.001320
Country_Singapore               0.001175
Country_Hong Kong               0.001010
Country_Channel Islands         0.000824
Country_Spain                   0.000630
Country_Lithuania               0.000621
Country_Greece                  0.000611
Country_Israel                  0.000535
Country_Austria                 0.000514
Country_Lebanon                 0.000468
Country_Brazil                  0.000356
Country_Italy                   0.000291
Country_United A

In [24]:
df_cleaned.to_csv("cleaned_retail_data.csv", index=False)

## Conclusion

In this notebook, we performed comprehensive data preparation on the online retail dataset. The process included loading, exploring, cleaning, feature engineering, and transforming the data using scaling techniques. The cleaned and preprocessed dataset is now ready for clustering analysis in the next step.

This preparation lays the groundwork for identifying meaningful customer segments, which will help the business improve targeting and personalization strategies.
